In [1]:
russian_letters = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"

In [2]:
import vk_api
import re
# import sys
import rus_preprocessing_udpipe
from importlib import reload
# reload(rus_preprocessing_udpipe)
# import gensim

import pandas as pd
import numpy as np

from stop_words import get_stop_words

from simple_elmo import ElmoModel

In [3]:
# vk_session = vk_api.VkApi('+number', 'password')
# vk_session.auth()

In [4]:
# vk_session

### Get groups texts

In [5]:
def get_groups_text(user_id, used_groups_limit, used_posts_num, vk_session):
    groups = get_user_groups(user_id, vk_session)
    groups_text, popular_words = get_groups_posts_text(groups, used_groups_limit, used_posts_num, vk_session)
    return groups_text, popular_words


def get_user_groups(user_id, vk_session):
    return vk_session.method("groups.get", values={"user_id": user_id})["items"]


def get_groups_posts_text(groups, used_groups_limit, used_posts_num, vk_session):
#     global groups_texts
    groups_texts = _raw_groups_texts(groups, used_groups_limit, used_posts_num, vk_session)
    russian_texts, popular_words = extract_russian_words(groups_texts)
    
    return russian_texts, popular_words
        
def _raw_groups_texts(groups, used_groups_limit, used_posts_num, vk_session):
    groups_texts = []

    for idx, group in enumerate(groups[:used_groups_limit]):
        try:
            group_wall = vk_session.method("wall.get", values={"owner_id": -group})
            posts_texts = get_wall_texts(group_wall, used_posts_num)
            posts_texts = " ".join(posts_texts)
            groups_texts.append(posts_texts)

        except vk_api.ApiError:
            print("Приватный паблик")
            
    return groups_texts

def extract_russian_words(groups_texts):
    stop_words = get_stop_words("ru")
    
#     concatenated_texts = " ".join(groups_texts)
    russian_texts = keep_russian_letters(groups_texts)
    
    for idx, text in enumerate(russian_texts):
        filtered_words = [word for word in text.split() if word not in stop_words]
        filtered_russian_text = " ".join(filtered_words)
        russian_texts[idx] = filtered_russian_text
    
    popular_words = get_popular_words(russian_texts)
    
    russian_texts = rus_preprocessing_udpipe.run(russian_texts)
    return russian_texts, popular_words

def get_wall_texts(wall, limit):
    posts = wall["items"]
    posts_texts = []
    for post in posts[:limit]:
        posts_texts.append(post["text"])
        
    return posts_texts

def keep_russian_letters(texts):
    for idx, text in enumerate(texts):
        russian_text = "".join(re.findall(f"[{russian_letters}\s]", text))
        russian_text = re.sub("\s", " ", russian_text)
        texts[idx] = russian_text
    
    return texts

def get_popular_words(texts):
    concatenated_texts = " ".join(texts)
    groups_words = pd.Series(concatenated_texts.split())
    counts = groups_words.value_counts().head(50)
    return counts[counts.index.str.len() > 3]

In [6]:
# %%time
# groups_texts, popular_groups_words = get_groups_text(USER_ID)
# groups_texts[:2]

In [7]:
# popular_groups_words

### get user wall texts

In [8]:
def get_user_text(user_id, vk_session):
    user_wall = vk_session.method("wall.get", values={"owner_id": user_id})
    posts_texts = get_wall_texts(user_wall, limit=5)
    print(posts_texts)
    russian_texts, popular_words = extract_russian_words(posts_texts)
    return russian_texts, popular_words

In [9]:
# %%time
# user_texts, popular_user_words = get_user_text(USER_ID)
# user_texts[:5]

### embeddings

In [10]:
# %%time

# model = ElmoModel()
# model.load("196.zip", max_batch_size=5)

In [11]:
# len(groups_texts)

In [12]:
# %%time
# groups_embedding = np.zeros(1024)
# step = 10

# for text_start_idx in range(0, len(groups_texts), step):
#     text_end_idx = text_start_idx + step
#     embeddings = model.get_elmo_vector_average(groups_texts[text_start_idx: text_end_idx])
#     mean_embedding = embeddings.mean(axis=0)
#     groups_embedding += mean_embedding
#     del embeddings, mean_embedding
    
# groups_embedding /= ((len(groups_texts) + step - 1) // step)

In [13]:
# groups_embedding.shape

In [14]:
def build_embeddings(user_id, vk_login, vk_password, elmo_model, used_groups_limit, used_posts_num):
    vk_session = vk_api.VkApi(vk_login, vk_password)
    vk_session.auth()
    
    groups_texts, popular_groups_words = get_groups_text(user_id, used_groups_limit, used_posts_num, vk_session)
    
    
    groups_embedding = np.zeros(1024)
    step = 10

    for text_start_idx in range(0, len(groups_texts), step):
        text_end_idx = text_start_idx + step
        embeddings = elmo_model.get_elmo_vector_average(groups_texts[text_start_idx: text_end_idx])
        mean_embedding = embeddings.mean(axis=0)
        groups_embedding += mean_embedding
        del embeddings, mean_embedding

    groups_embedding /= ((len(groups_texts) + step - 1) // step)
    
    return groups_embedding


In [15]:
elmo_path = "196.zip"
elmo_batch = 5
USED_POSTS_NUM = 1
used_groups_limit = 10
USER_ID = "gushchin_d"
model = ElmoModel()
model.load(elmo_path, max_batch_size=elmo_batch)

embedding = build_embeddings(USER_ID, "+number", "password&R", elmo_model, used_groups_limit, used_posts_num=USED_POSTS_NUM)


Loading the model...
Processing input...
2020-11-29 02:45:30,319 : INFO : Loading model from 196.zip...
2020-11-29 02:45:30,320 : INFO : 
            Assuming the model is a ZIP archive downloaded from the NLPL vector repository.
            Loading a model from a ZIP archive directly is slower than from the extracted files,
            but does not require additional disk space
            and allows to load from directories without write permissions.
            
2020-11-29 02:45:30,384 : INFO : We will cache the vocabulary of 100 tokens.


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


2020-11-29 02:46:50,477 : WARNING : From /home/gldsn/.local/share/virtualenvs/innohack-citadel-YNBG4hQA/lib/python3.8/site-packages/simple_elmo/model.py:529: LSTMCell.__init__ (from tensorflow.python.keras.layers.legacy_rnn.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


2020-11-29 02:46:50,505 : WARNING : From /home/gldsn/.local/share/virtualenvs/innohack-citadel-YNBG4hQA/lib/python3.8/site-packages/simple_elmo/model.py:570: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Instructions for updating:
Please use `layer.add_weight` method instead.


2020-11-29 02:46:50,744 : WARNING : From /home/gldsn/.local/share/virtualenvs/innohack-citadel-YNBG4hQA/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:962: Layer.add_variable (from tensorflow.python.keras.engine.base_layer_v1) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2020-11-29 02:46:58,090 : WARNING : From /home/gldsn/.local/share/virtualenvs/innohack-citadel-YNBG4hQA/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:970: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
2020-11-29 02:48:15,535 : INFO : Warming up ELMo on 5 sentences...
2020-11-29 02:48:30,406 : INFO : Warming up finished.
2020-11-29 02:48:30,434 : INFO : Texts in the current batch: 5
2020-11-29 02:48:44,748 : INFO : Texts in the current batch: 5


In [18]:
embedding

array([-0.0235609 ,  0.00990808, -0.03002209, ..., -0.01308269,
       -0.00175042, -0.03019912])

### LEGACY

In [16]:
# # категории подписок
# subscriptions = vk_session.method("users.getSubscriptions", values={"user_id": USER_ID})["groups"]["items"]

# for category_info in group_catalog_info["categories"]:
#     if category_info["name"] not in ["Популярные", "Бренды"]:
#         category = vk_session.method("groups.getCatalog", values={"category_id": category_info["id"]})
#         print("category", category_info["id"])
#         print("count", category["count"])
#         print(category)

# subscriptions = vk_session.method("users.getSubscriptions", values={"user_id": USER_ID})["groups"]["items"]

In [17]:
# subscriptions = vk_session.method("users.getSubscriptions", values={"user_id": USER_ID})["groups"]["items"]